### Libraries

In [1]:
# import libraries

import cv2
import os
import json
import numpy as np

### Data directory

In [2]:
# main paths

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Input file

In [11]:
# open an image

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 49

original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
original_image = cv2.resize(original_image, (0, 0), fx = 0.15, fy = 0.15)

cv2.imshow('Original', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# gray and hsv versions

gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

cv2.imshow('Gray', gray_image)
cv2.imshow('HSV', hsv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
# check if a box is inside another

def is_inside(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    return x2 <= x1 and y2 <= y1 and x2 + w2 >= x1 + w1 and y2 + h2 >= y1 + h1

### Background medium testing

In [159]:
def update_hue_tolerance(*args):
    global hue_tolerance
    hue_tolerance = cv2.getTrackbarPos("Hue", window_name)
    process_image()

def update_saturation_tolerance(*args):
    global saturation_tolerance
    saturation_tolerance = cv2.getTrackbarPos("Saturation", window_name)
    process_image()

def update_value_tolerance(*args):
    global value_tolerance
    value_tolerance = cv2.getTrackbarPos("Value", window_name)
    process_image()

def process_image():
    lower_limit_0 = max(background_color[0] - hue_tolerance, 0)
    upper_limit_0 = min(background_color[0] + hue_tolerance, 180)
    lower_limit_1 = max(background_color[1] - saturation_tolerance, 0)
    upper_limit_1 = min(background_color[1] + saturation_tolerance, 255)
    lower_limit_2 = max(background_color[2] - value_tolerance, 0)
    upper_limit_2 = min(background_color[2] + value_tolerance, 255)
    
    lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
    upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])

    blur = cv2.GaussianBlur(hsv_image, (3, 3), 0)
    
    gray_mask = cv2.inRange(blur, lower_background, upper_background)
    non_gray_mask = cv2.bitwise_not(gray_mask)
    non_gray_mask = cv2.erode(non_gray_mask, None, iterations=3)
    
    image = original_image.copy()
    result = cv2.bitwise_and(image, image, mask=non_gray_mask)
    
    contours, _ = cv2.findContours(non_gray_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    img_with_boxes = original_image.copy()

    removed = []
    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        inside_other_box = False
        for j in range(i+1, len(contours)):
            prev_x, prev_y, prev_w, prev_h = cv2.boundingRect(contours[j])
            if is_inside((x, y, w, h), (prev_x, prev_y, prev_w, prev_h)):
                inside_other_box = True
                break
        if not inside_other_box:
            cv2.rectangle(img_with_boxes, (x, y), (x + w, y + h), (255, 255, 0), 2)
        else:
            removed.append(i)
    contours = [contour for i, contour in enumerate(contours) if i not in removed]

    combined_image = np.hstack((result, image))
    cv2.imshow(window_name, combined_image)

hsv = hsv_image.copy()

hsv_flattened = hsv.reshape((-1, 3))
hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]
background_color = np.median(hsv_flattened, axis=0)

hue_tolerance = 100
saturation_tolerance = 100
value_tolerance = 100

window_name = "TEST"
cv2.namedWindow(window_name)

cv2.createTrackbar("Hue", window_name, hue_tolerance, 180, update_hue_tolerance)
cv2.createTrackbar("Saturation", window_name, saturation_tolerance, 255, update_saturation_tolerance)
cv2.createTrackbar("Value", window_name, value_tolerance, 255, update_value_tolerance)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()


### Canny testing

In [97]:
image = original_image.copy()
gray = gray_image.copy()
hsv = hsv_image.copy()

gaussian = cv2.GaussianBlur(gray, (7, 7), 0)

reshaped_image = gaussian.reshape((-1, 1))
reshaped_image = np.float32(reshaped_image)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
k = 50

_, label, center = cv2.kmeans(reshaped_image, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

center = np.uint8(center)
kmeans = center[label.flatten()]
kmeans = kmeans.reshape((gaussian.shape))

max_value = 255
threshold_low_value = 100
threshold_high_value = 200
window_name = "TEST"

cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

def find_and_draw_contours(low, high, img):
    canny_image = cv2.Canny(img, low, high)
    contours, _ = cv2.findContours(canny_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_with_boxes = original_image.copy()
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    removed = []
    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        inside_other_box = False
        for j in range(i+1, len(contours)):
            prev_x, prev_y, prev_w, prev_h = cv2.boundingRect(contours[j])
            if is_inside((x, y, w, h), (prev_x, prev_y, prev_w, prev_h)):
                inside_other_box = True
                break
        if not inside_other_box:
            cv2.rectangle(img_with_boxes, (x, y), (x + w, y + h), (255, 255, 0), 2)
        else:
            removed.append(i)
    contours = [contour for i, contour in enumerate(contours) if i not in removed]

    canny_image_bgr = cv2.cvtColor(canny_image, cv2.COLOR_GRAY2BGR)
    combined_image = np.hstack((canny_image_bgr, img_with_boxes))
    cv2.imshow(window_name, combined_image)

def alter_low_value(*args):
    global threshold_low_value
    threshold_low_value = args[0]
    find_and_draw_contours(threshold_low_value, threshold_high_value, kmeans)

def alter_high_value(*args):
    global threshold_high_value
    threshold_high_value = args[0]
    find_and_draw_contours(threshold_low_value, threshold_high_value, kmeans)

cv2.createTrackbar("Low", window_name, threshold_low_value, max_value, alter_low_value)
cv2.createTrackbar("High", window_name, threshold_high_value, max_value, alter_high_value)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Lego detection

In [167]:
def median_background(hsv):
    hsv_flattened = hsv.reshape((-1, 3))
    hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]
    background_color = np.median(hsv_flattened, axis=0)
    return background_color

def limits(background_color, hue_tolerance, saturation_tolerance, value_tolerance):
    lower_limit_0 = max(background_color[0] - hue_tolerance, 0)
    upper_limit_0 = min(background_color[0] + hue_tolerance, 180)
    lower_limit_1 = max(background_color[1] - saturation_tolerance, 0)
    upper_limit_1 = min(background_color[1] + saturation_tolerance, 255)
    lower_limit_2 = max(background_color[2] - value_tolerance, 0)
    upper_limit_2 = min(background_color[2] + value_tolerance, 255)
    lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
    upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])
    return lower_background, upper_background

def background_median(hsv):
    background_color = median_background(hsv)
    hue_tolerance = 20
    saturation_tolerance = 90
    value_tolerance = 90
    lower_background, upper_background = limits(background_color, hue_tolerance, saturation_tolerance, value_tolerance)
    blur = cv2.blur(hsv, (5, 5))
    gray_mask = cv2.inRange(blur, lower_background, upper_background)
    non_gray_mask = cv2.bitwise_not(gray_mask)
    non_gray_mask = cv2.erode(non_gray_mask, None, iterations=3)
    return non_gray_mask

def kmeans_segments(image):
    reshaped_image = image.reshape((-1,1))
    reshaped_image = np.float32(reshaped_image)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    k = 50
    _, label, center = cv2.kmeans(reshaped_image, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    result = center[label.flatten()]
    result = result.reshape((image.shape))
    return result

def canny_edges(gray):
    gaussian = cv2.GaussianBlur(gray, (7, 7), 0)
    #kmeans = kmeans_segments(gaussian)
    canny = cv2.Canny(gaussian, 100, 200)
    return canny

In [171]:
image = original_image.copy()
gray = gray_image.copy()
hsv = hsv_image.copy()

bg_median = background_median(hsv)
canny = canny_edges(gray)

combined_mask = cv2.bitwise_or(bg_median, canny)
result = cv2.bitwise_and(image, image, mask=combined_mask)

contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

removed = []
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    inside_other_box = False
    for j in range(i+1, len(contours)):
        prev_x, prev_y, prev_w, prev_h = cv2.boundingRect(contours[j])
        if is_inside((x, y, w, h), (prev_x, prev_y, prev_w, prev_h)):
            inside_other_box = True
            break
    if not inside_other_box and w * h > 400:
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
    else:
        removed.append(i)
contours = [contour for i, contour in enumerate(contours) if i not in removed]

num_lego_pieces = len(contours)
print("Number of LEGO pieces detected:", num_lego_pieces)

cv2.imshow('Image', image)
cv2.imshow('Background median', bg_median)
cv2.imshow('Canny', canny)
cv2.imshow('Combined Mask', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of LEGO pieces detected: 15
